In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import nltk

In [2]:
df = pd.read_csv('../../Datasets/cleaned_customer.csv')

In [3]:
df.head()

,reviewer_id,listing_id,date,reviewer_name,comments,name,description,neighborhood_overview,host_id,host_since,...,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,reviews_per_month,nearby_sites_rating,nearest_station_dist,nearest_site_dist,nearest_site_rating
0,8557223,49091,2013-10-21,Jared,Fran was absolutely gracious and welcoming. Ma...,COZICOMFORT LONG TERM STAY ROOM 2,This is Room No. 2.(available after Jan 15 201...,NaN,266763,2010-10-20,...,10.0,8.0,8.0,f,2,0.02,0.995731,0.617762,4.295315,4.595264
1,1356099,50646,2014-04-18,James,A comfortable room in a smart condo developmen...,Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,...,10.0,9.0,9.0,f,1,0.30,1.269606,0.399696,2.039236,4.654423
2,15222393,50646,2014-06-05,Welli,Stayed over at Sujatha's house for 3 good nigh...,Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,...,10.0,9.0,9.0,f,1,0.30,1.269606,0.399696,2.039236,4.654423
3,5543172,50646,2014-07-02,Cyril,It's been a lovely stay at Sujatha's. The room...,Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,...,10.0,9.0,9.0,f,1,0.30,1.269606,0.399696,2.039236,4.654423
4,817532,50646,2014-07-08,Jake,"We had a great experience. A nice place, an am...",Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,2010-09-08,...,10.0,9.0,9.0,f,1,0.30,1.269606,0.399696,2.039236,4.654423


In [9]:
df.comments.isna().value_counts()

False    155250
Name: comments, dtype: int64

In [10]:
df.comments.fillna('',inplace=True)

### 1. Tokenize the text and extract most frequent words

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

def tokenizer(text):
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

vectorizer = TfidfVectorizer(stop_words='english',
                             ngram_range=(1,2),
                             tokenizer = tokenizer,
                             max_df=0.5,
                             min_df=0.0001,
                             max_features=10000)
X = vectorizer.fit_transform(df.comments)

Version: ImageMagick 7.0.10-58 Q16 x86_64 2021-01-16 https://imagemagick.org
Copyright: © 1999-2021 ImageMagick Studio LLC
License: https://imagemagick.org/script/license.php
Features: Cipher DPC HDRI Modules OpenMP(4.5) 
Delegates (built-in): bzlib freetype gslib heic jng jp2 jpeg lcms lqr ltdl lzma openexr png ps tiff webp xml zlib
Usage: import [options ...] [ file ]

Image Settings:
  -adjoin              join images into a single multi-image file
  -border              include window border in the output image
  -channel type        apply option to select image channels
  -colorspace type     alternate image colorspace
  -comment string      annotate image with comment
  -compress type       type of pixel compression when writing the image
  -define format:option
                       define one or more image format options
  -density geometry    horizontal and vertical density of the image
  -depth value         image depth
  -descend             obtain image by descending windo

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/hanjiatong/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.8/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.8/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.8/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1108)>


False

In [48]:
print(vectorizer.get_feature_names())

['00', '10', '10 10', '10 12', '10 15', '10 15min', '10 15mins', '10 days', '10 min', '10 mins', '10 minute', '10 minutes', '10 recommend', '100', '100 recommend', '100m', '10am', '10min', '10min walk', '10mins', '10mins walk', '10pm', '10분', '11', '11 days', '11am', '11pm', '12', '12 days', '12pm', '13', '13 days', '14', '15', '15 20', '15 days', '15 min', '15 mins', '15 minute', '15 minutes', '15min', '15min walk', '15mins', '15mins walk', '16', '16 days', '17', '18', '19', '1am', '1km', '1st', '1st floor', '20', '20 30', '20 days', '20 min', '20 mins', '20 minute', '20 minutes', '200', '200m', '20min', '20mins', '21', '22', '23', '24', '24 hour', '24 hours', '24 hr', '24 hrs', '24h', '24hours', '24hr', '24hr supermarket', '24hrs', '25', '25 minutes', '26', '27', '28', '2am', '2min', '2mins', '2nd', '2nd floor', '2nd stay', '2nd time', '2pm', '30', '30 min', '30 mins', '30 minutes', '300', '30am', '30min', '30mins', '30pm', '31', '313', '32', '35', '36', '3am', '3min', '3mins', '3min

### 1. Using Afinn for sentiment analysis
Afinn measures the sentiment score (-5 to 5) for each single word. But it does not consider word combinations.

In [4]:
from afinn import Afinn

In [ ]:
# Score for an empty string
afinn.score('')

In [12]:
afinn = Afinn()
afinn_score = np.array([afinn.score(text) for text in df.comments])

In [28]:
pd.DataFrame(afinn_score).describe()

,0
count,155250.000000
mean,6.907124
std,7.008292
min,-62.000000
25%,2.000000
50%,6.000000
75%,10.000000
max,131.000000


In [43]:
df[afinn_score < -30].comments

765       Ich war 3 Wochen lang bei Familie Woon unterge...
2563      - The ventilation in the room is horrible. The...
2977      Das Top-Erlebnis über Weihnachten in Singapur....
5281      Wir haben Mitte August 3 Nächte bei Darrick On...
6048      Wir haben uns die Unterkunft nach den Bildern ...
36042     Die Kommunikation mit Gary war sehr gut und de...
40344     Zu allererst muss man die Gastfreundschaft von...
49830     Wir waren alles in allem enttäuscht von unsere...
52844     Wenn man die Straße, in der das Apartment lieg...
100158    Wir waren im Juni 2018 für 7 Tage in Singapur ...
118156    Freundlicherweise durften wir unsere Koffer fr...
134859    Ansich ist diese Art von Übernachtung eine lus...
146874    Die Unterkunft an sich ist in Ordnung. Im Prin...
151631    Book at your own risk. Superhost? More like su...
Name: comments, dtype: object

#### Discovery: It seems that most of the extremely low-rated comments are written in German. But Afinn hasn't yet been equipped to understand German words.

In [44]:
df[afinn_score > 70].comments

422      What you see is what you got.. That is what i ...
2296     Hello everyone,\n\nYuan's Airbnb is excellent ...
11893    I have enjoyed my stay there Hospitality and F...
19515    It was a great experience for me staying at Na...
22174    I stayed with Suan Khim and her wonderful fami...
25745    "Great friendly hospitality. Clean and peacefu...
26298    I had a wonderful week staying at Felix's plac...
33338    Hi there , I'm very lucky to get a opportunity...
48966    Our family of 5 enjoyed a wonderful 6 nights i...
90524    Ian is very serious in being the best host. A ...
95698    I had the pleasure of staying with Nana and Lu...
96757    My stay from 22th November to 1st December was...
96761    Staying in this room at Rain and Sarah's villa...
Name: comments, dtype: object

#### There seems to be no much problem with extremely highly rated comments.

### 2. Using Textblob for sentiment analysis

#### Features of Textblob

1. Textblob measures polarity score between -1 and 1, negative to postive. 
2. Ignore the words it does not know.
3. Can also measure the subjectivity score.